## Scraping Multiple Thread Tribunews

In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
import re
from urllib.request import Request, urlopen

In [10]:
jumlah_index = 1
threads_link = []
links = []

In [11]:
def scrape_links(page_number):
    url = f"https://www.tribunnews.com/2023/09?page={page_number}"
    try:
        response = requests.get(url, headers={'User-Agent': 'Chrome/117.0.0.0'})
        response.raise_for_status()  # Raise an exception for HTTP errors
        soup = BeautifulSoup(response.text, 'html.parser')
        articles = soup.find_all('li', {"class": "ptb15"})
        page_links = []
        for article in articles:
            header = article.find('h3', {"class": "fbo f16"})
            link = header.find('a')['href']
            links.append(link)
        print(f"Scraped {len(page_links)} links from page {page_number}")
        return page_links
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching data from {url}: {e}")   

In [12]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 0 links from page 1
Total Links: 20


In [13]:
results = []
def scrape_url(url):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Judul Berita
            title_elem = soup.find('h1', {"id": "arttitle"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"id": "penulis"})
            if author_elem:
                penulis = author_elem.find('a')
                author_text =penulis.text.strip()
            else:
                author_text = "Author not found"
               # Editor Berita
            editor_elem = soup.find('div', {"id": "editor"})
            
            if editor_elem:
                editors = editor_elem.find('a')
                editor_text = editors.text.strip()
            else:
                editor_text = "editor not found" 
            
            # tanggal berita
            date_elem = soup.find('time', {"class": "grey"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            # category_elements = soup.find_all('span',{"itemprop": "name"})
            # if category_elements:
            #     category_text= category_elements[2].get_text()
            # else:
            category_text = "Category not found"
            ## Content Berita
            body_elem = soup.find('div', {"class": "side-article txt-article multi-fontsize"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
              content_text ="Content not found"
            nama_berita_match = re.search(r'https://www\.(\w+)\.com/', url)
            if nama_berita_match:
                nama_berita = nama_berita_match.group(1)
            else:
                nama_berita = "Nama_berita not found"
            results.append({'title': title_text,
                            'author' : author_text,
                            'editor' : editor_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'nama_berita' : nama_berita,
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [14]:
threads = []
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [15]:
df = pd.DataFrame(results)
print("hasil data scrapping",len(results))
df.head(10)

hasil data scrapping 20


,title,author,editor,category,date,content,nama_berita,link
0,Megawati Buka Rakernas IV PDI-P Hari Ini: Joko...,Author not found,Wahyu Aji,Category not found,"Jumat, 29 September 2023 08:32 WIB","TRIBUNNEWS.COM, JAKARTA - Ketua Umum PDI Perju...",tribunnews,https://www.tribunnews.com/nasional/2023/09/29...
1,Cara Beli dan Pasang E-Meterai di meteraionlin...,Widya Lisfianti,Whiesa Daniswara,Category not found,"Jumat, 29 September 2023 08:29 WIB","TRIBUNNEWS.COM - Hari ini, Jumat (29/9/2023) m...",tribunnews,https://www.tribunnews.com/nasional/2023/09/29...
2,"Ramalan Zodiak Karier Hari Ini, Jumat 29 Septe...",Enggar Kusuma Wardani,Nuryanti,Category not found,"Jumat, 29 September 2023 08:20 WIB",TRIBUNNEWS.COM - Inilah ramalan zodiak karier ...,tribunnews,https://www.tribunnews.com/lifestyle/2023/09/2...
3,Nilai Ambang Batas PPPK Guru 2023: Seleksi Kom...,Yunita Rahmayanti,Nuryanti,Category not found,"Jumat, 29 September 2023 08:17 WIB",TRIBUNNEWS.COM - Berikut ini nilai ambang bata...,tribunnews,https://www.tribunnews.com/nasional/2023/09/29...
4,Kereta Cepat China Fuxing G9801 Lintasi Jalur ...,Mikael Dafit Adi Prasetyo,Choirul Arifin,Category not found,"Jumat, 29 September 2023 08:36 WIB","Laporan Wartawan Tribunnews.com, Mikael Dafit ...",tribunnews,https://www.tribunnews.com/bisnis/2023/09/29/k...
5,"Anies-Cak Imin Bertemu Rizieq Shihab, FPI: Jal...",Author not found,Dewi Agustina,Category not found,"Jumat, 29 September 2023 08:24 WIB","TRIBUNNEWS.COM, JAKARTA - Pasangan bakal capre...",tribunnews,https://www.tribunnews.com/mata-lokal-memilih/...
6,20 Ucapan Hari Jantung Sedunia 2023 dalam Baha...,Yurika Nendri Novianingsih,Whiesa Daniswara,Category not found,"Jumat, 29 September 2023 08:24 WIB","TRIBUNNEWS.COM - Tepat pada hari ini, Jumat (2...",tribunnews,https://www.tribunnews.com/nasional/2023/09/29...
7,Polres Gianyar Periksa 2 Tersangka Tragedi Lif...,Author not found,Theresia Felisiani,Category not found,"Jumat, 29 September 2023 08:37 WIB","TRIBUNNEWS.COM, BALI - Polres Gianjar hari ini...",tribunnews,https://www.tribunnews.com/regional/2023/09/29...
8,"Tegur Tetangganya yang Pesta Miras, Kepala Dri...",Author not found,Theresia Felisiani,Category not found,"Jumat, 29 September 2023 08:24 WIB","TRIBUNNEWS.COM, DENPASAR - Seorang driver ojek...",tribunnews,https://www.tribunnews.com/regional/2023/09/29...
9,Matty Healy Umumkan Band 1975 akan Hiatus Tanp...,Andari Wulan Nugrahani,Salma Fenty,Category not found,"Jumat, 29 September 2023 08:17 WIB","TRIBUNNEWS.COM - Vokalis band The 1975, Maty H...",tribunnews,https://www.tribunnews.com/seleb/2023/09/29/ma...


save_date

In [16]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'../tempat_simpan_nasional/tribunnews_{current_datetime}.xlsx'
# df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')